In [491]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

In [237]:
train = pd.read_csv(os.path.join('..', 'data', 'proc', 'train.csv'))
test = pd.read_csv(os.path.join('..', 'data', 'proc', 'test.csv'))

In [674]:
item_meta = pd.read_csv(os.path.join('..', 'data', 'proc', 'item_metadata.csv'))

In [238]:
# 행동 순서를 파악하기 위한 시간 설정
train['dt'] = train['dt'].astype('datetime64')
test['dt'] = test['dt'].astype('datetime64')

In [254]:
item = item_meta['item_id'].reset_index().rename(columns = {'index' : 'item_idx'})

# click_out만 뽑기

In [358]:
tr = train.loc[train['action_type'] == 'clickout item'].sort_values(['user_id', 'timestamp'])[['user_id', 'reference', 'timestamp']]
te = test.loc[test['action_type'] == 'clickout item'].sort_values(['user_id', 'timestamp'])[['user_id', 'reference', 'timestamp']]

In [359]:
te_notnull = te.loc[te['reference'].notnull()]

In [403]:
interactions_click = pd.concat([tr, te_notnull], axis = 0)

In [404]:
interactions_click.shape

(885469, 3)

In [405]:
interactions_click = interactions_click.sort_values(['user_id', 'timestamp']).reset_index(drop = True)[['user_id', 'reference']]

In [311]:
user = interactions_click['user_id'].drop_duplicates().reset_index(drop = True).reset_index().rename(columns = {'index' : 'user_idx'})

In [406]:
interactions_click = (interactions_click
                      .merge(user, on = 'user_id', how = 'left')
                      .merge(item, left_on = 'reference', right_on = 'item_id', how = 'left')
                      .drop(['user_id', 'reference', 'item_id'], axis = 1)
                     )

In [407]:
interactions_click = interactions_click.loc[interactions_click['item_idx'].notnull()]

In [408]:
interactions_click.to_csv('trv_click.txt', index=False, header=None, sep="\t")

In [414]:
interactions_click['item_idx'].astype(int).max()

927134

In [333]:
idx2user = user.drop('user_idx', axis = 1).to_dict()['user_id']
user2idx = {j:i for i,j in idx2user.items()}

In [337]:
idx2item = item.drop('item_idx', axis = 1).to_dict()['item_id']
item2idx = {j:i for i,j in idx2item.items()}

In [339]:
import joblib
data = {
    'idx2user' : idx2user,
    'user2idx' : user2idx,
    'idx2item' : idx2item,
    'item2idx' : item2idx
}
joblib.dump(data, 'dic4sas.pkl')

['dic4sas.pkl']

# interaction 모두 뽑기

In [340]:
# reference가 item으로만 이뤄진 interaction 뽑기
import re
def findNum(x):
    return re.sub(r'[ㄱ-ㅎ가-힣a-zA-Z,]', '@',x)

In [341]:
train['reference2'] = train['reference'].apply(lambda x : findNum(x))
train['reference2'] = train['reference2'].apply(lambda x : None if '@' in x else x)
tr = train.loc[train['reference2'].notnull()].sort_values(['user_id', 'timestamp'])[['user_id', 'reference', 'timestamp']]

In [342]:
test = test.loc[test['reference'].notnull()]
test['reference2'] = test['reference'].apply(lambda x : findNum(x))
test['reference2'] = test['reference2'].apply(lambda x : None if '@' in x else x)
te = test.loc[test['reference2'].notnull()].sort_values(['user_id', 'timestamp'])[['user_id', 'reference', 'timestamp']]

In [351]:
interactions_item = pd.concat([tr, te], axis = 0).sort_values(['user_id', 'timestamp']).sort_values(['user_id', 'timestamp']).drop('timestamp', axis = 1)

In [352]:
interactions_item = interactions_item.merge(user, on = 'user_id', how = 'left').merge(item, left_on = 'reference', right_on = 'item_id', how = 'left').drop(['user_id', 'reference', 'item_id'], axis = 1)

In [353]:
interactions_item

,user_idx,item_idx
0,0,99559
1,0,99559
2,0,99559
3,0,99559
4,0,99559
...,...,...
8976876,400287,728608
8976877,400287,728608
8976878,400287,728608
8976879,400287,728608


In [419]:
interactions_item = interactions_item.loc[interactions_item['item_idx'].notnull()]

In [420]:
interactions_item.to_csv('trv_item.txt', index=False, header=None, sep="\t")

In [415]:
item

,item_idx,item_id
0,0,5101
1,1,5416
2,2,5834
3,3,5910
4,4,6066
...,...,...
927137,927137,11199990
927138,927138,11236100
927139,927139,11260526
927140,927140,11263712


# seoul 호텔 검색 세선

In [421]:
train = pd.read_csv(os.path.join('..', 'data', 'proc', 'seoul_train.csv'))
test = pd.read_csv(os.path.join('..', 'data', 'proc', 'seoul_test.csv'))

In [432]:
tr = train.loc[train['action_type'] == 'clickout item', ['user_id','reference']]

In [433]:
te = test.loc[test['action_type'] == 'clickout item', ['user_id','reference']]

In [437]:
df = pd.concat([tr, te], axis = 0)

In [447]:
df['user_id'].nunique()

1408

In [445]:
(df.groupby('user_id')['reference'].count() > 2).sum()

358

In [457]:
# 서울을 검색한 기록이 있는 user
idx = list(set(list(train['user_id'].unique()) + list(test['user_id'].unique())))

In [486]:
train = pd.read_csv(os.path.join('..', 'data', 'proc', 'raw_train.csv'))
test = pd.read_csv(os.path.join('..', 'data', 'proc', 'raw_test.csv'))

In [487]:
log_all = pd.concat([train,test], axis = 0)

In [488]:
log_all.index = log_all['user_id']

In [514]:
len(idx)

1581

In [530]:
log_all.loc[log_all['city'] == 'Seoul, South Korea'].reset_index(drop = True).to_excel('./seoul_session.xlsx')

In [541]:
df = log_all.loc[log_all['city'] == 'Seoul, South Korea'].reset_index(drop = True)

In [543]:
df.loc[df['action_type'] == 'clickout item']

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
1,0L2TX0JNYVQ6,06e7c29170946,1541041870,2,clickout item,10091602,HK,"Seoul, South Korea",desktop,NaN,2802232|2733571|5477718|155374|155465|3549258|...,124|176|99|220|191|127|85|54|83|268|78|144|96|...
4,0L2TX0JNYVQ6,06e7c29170946,1541044151,5,clickout item,10091602,HK,"Seoul, South Korea",desktop,NaN,3549258|155465|155374|363046|3954788|4773608|3...,135|189|219|78|74|135|95|85|176|99|108|83|87|3...
7,0L2TX0JNYVQ6,06e7c29170946,1541044294,8,clickout item,1394008,HK,"Seoul, South Korea",desktop,NaN,2736998|9386776|7016734|2867638|797516|5677278...,249|122|110|81|81|42|74|109|62|91|83|104|78|87...
8,0L2TX0JNYVQ6,06e7c29170946,1541044324,9,clickout item,1394008,HK,"Seoul, South Korea",desktop,NaN,2736998|9386776|7016734|2867638|797516|5677278...,249|122|110|81|81|42|74|109|62|91|83|104|78|87...
9,0L2TX0JNYVQ6,06e7c29170946,1541044344,10,clickout item,5109850,HK,"Seoul, South Korea",desktop,NaN,5109850|3505382|377811|155374|4849224|3501452|...,59|52|66|219|104|95|85|69|125|176|99|83|30|268...
...,...,...,...,...,...,...,...,...,...,...,...,...
39922,QOVMYB65MO00,78ffdc618a4fd,1541557759,12,clickout item,NaN,JP,"Seoul, South Korea",desktop,Hotel|5 Star|4 Star|3 Star|Free WiFi (Combined),363046|155486|155348|844336|1176048|475296|475...,90|73|201|69|80|70|103|62|112|82|58|97|56|126|...
39924,CPLU2L7YFW64,650a3064ca06b,1541632281,2,clickout item,6833196,JP,"Seoul, South Korea",desktop,NaN,6833196|7350166|3491670|4841728|8974570|797516...,166|157|205|264|223|356|169|548|263|1215|53|27...
39925,CPLU2L7YFW64,650a3064ca06b,1541632355,3,clickout item,6833196,JP,"Seoul, South Korea",desktop,NaN,6833196|7350166|3491670|4841728|8974570|797516...,166|157|205|264|223|356|169|548|263|1215|53|27...
39931,CPLU2L7YFW64,650a3064ca06b,1541632819,9,clickout item,3475208,JP,"Seoul, South Korea",desktop,Hotel|Sort By Distance,10493016|3475208|5819970|4387254|4343730|21992...,132|145|121|190|179|174|116|180|175|137|176|17...


In [544]:
df.loc[df['action_type'] == 'clickout item', 'reference'].nunique(), df.loc[df['action_type'] == 'clickout item', 'user_id'].nunique()

(741, 1807)

In [568]:
df_click = df.loc[(df['action_type'] == 'clickout item') & (df['reference'].notnull())]

In [569]:
df_click = (df_click[['user_id', 'reference', 'timestamp']]
            .merge(df_click.groupby('user_id')['reference'].count()
                   .reset_index(), on = 'user_id', how = 'left')
           .sort_values(['user_id', 'timestamp']))

In [570]:
df_click

,user_id,reference_x,timestamp,reference_y
1002,002GA0R9935R,155461,1541434155,1
735,00CR4VNMYT1J,9196336,1541305950,1
2000,024L2YJV0SRX,2802232,1541419921,7
2001,024L2YJV0SRX,155989,1541419944,7
2002,024L2YJV0SRX,3132376,1541420056,7
...,...,...,...,...
1755,ZWFZVALX5VP1,363061,1541085518,2
1756,ZWFZVALX5VP1,363081,1541086238,2
126,ZYQUCE9LUM5W,3184625,1541304483,1
2208,ZZ323VWV4TOS,2733571,1541045593,2


In [571]:
df_click.loc[df_click['reference_y'] > 2, ['user_id', 'reference_x']]

,user_id,reference_x
2000,024L2YJV0SRX,2802232
2001,024L2YJV0SRX,155989
2002,024L2YJV0SRX,3132376
2003,024L2YJV0SRX,844336
2004,024L2YJV0SRX,363096
...,...,...
1802,ZJPPH5Z85CL3,6466284
1803,ZJPPH5Z85CL3,5169964
311,ZP00669TCBB6,363071
312,ZP00669TCBB6,7016734


In [572]:
# seoul 호텔을 검색해본 user 뽑기

In [576]:
idx = log_all.loc[log_all['city'] == 'Seoul, South Korea'].index.unique()

In [578]:
df2 = log_all.loc[idx].reset_index(drop = True)

In [590]:
df2_click = df2.loc[(df2['action_type'] == 'clickout item') & (df2['reference'].notnull())]

In [591]:
df2_click = (df2_click[['user_id', 'reference', 'timestamp', 'city']]
            .merge(df2_click.groupby('user_id')['reference'].count()
                   .reset_index(), on = 'user_id', how = 'left')
           .sort_values(['user_id', 'timestamp']))

In [593]:
df2_click.loc[df2_click['reference_y'] > 2, ['user_id', 'reference_x', 'city']]

,user_id,reference_x,city
2813,024L2YJV0SRX,2802232,"Seoul, South Korea"
2814,024L2YJV0SRX,155989,"Seoul, South Korea"
2815,024L2YJV0SRX,3132376,"Seoul, South Korea"
2816,024L2YJV0SRX,844336,"Seoul, South Korea"
2817,024L2YJV0SRX,363096,"Seoul, South Korea"
...,...,...,...
3374,ZOPTENE0D3K2,103493,"Kobe, Japan"
3375,ZOPTENE0D3K2,502971,"Beppu, Japan"
668,ZP00669TCBB6,363071,"Seoul, South Korea"
669,ZP00669TCBB6,7016734,"Seoul, South Korea"


In [604]:
idx2item = df2_click.loc[(df2_click['reference_y'] > 2) & (df2_click['city'] == 'Seoul, South Korea'), ['reference_x', 'city']].drop_duplicates().reset_index(drop = True)['reference_x'].to_dict()

In [623]:
max(idx2item.keys())

620

In [609]:
t = df2_click.loc[(df2_click['reference_y'] > 2) & (df2_click['city'] != 'Seoul, South Korea'), ['reference_x', 'city']].drop_duplicates()['reference_x']

In [617]:
len(t)

543

In [621]:
# 서울 호텔은 ~620까지, 621~1163 까지 다른나라 호텔
t.index = np.arange(621, 621+543)

In [628]:
idx2item.update(t)

In [664]:
final = df2_click.loc[df2_click['reference_y'] > 2, ['user_id', 'reference_x']]

In [639]:
idx2user = final['user_id'].drop_duplicates().reset_index(drop = True).to_dict()

In [648]:
user2idx = {q : p for p,q in idx2user.items()}

In [659]:
final

,user_id,reference_x
2813,024L2YJV0SRX,2802232
2814,024L2YJV0SRX,155989
2815,024L2YJV0SRX,3132376
2816,024L2YJV0SRX,844336
2817,024L2YJV0SRX,363096
...,...,...
3374,ZOPTENE0D3K2,103493
3375,ZOPTENE0D3K2,502971
668,ZP00669TCBB6,363071
669,ZP00669TCBB6,7016734


In [666]:
final = (final.merge(pd.json_normalize(user2idx).T.reset_index(), left_on = 'user_id', right_on = 'index', how = 'left')
         .merge(pd.json_normalize(idx2item).T.reset_index(), left_on = 'reference_x', right_on = 0, how = 'left'))

In [669]:
final[['0_x', 'index_y']].to_csv('./test.txt', header = None, index = False)

# 모든 세션 고려하되, 출력은 서울 호텔로

In [673]:
# 전체데이터에서 user는 94만명
# 전체데이터에서 item은 43만개
log_all['user_id'].nunique(), log_all['reference'].nunique()

(948041, 434873)

In [690]:
# item2idx 만들기
# 서울 호텔은 따로 빼기
seoul_item = log_all.loc[log_all['city'] == 'Seoul, South Korea', 'reference'].drop_duplicates().reset_index(drop = True).reset_index()

In [686]:
item_meta['item_id'] = item_meta['item_id'].astype(str)

In [688]:
item_meta.shape

(927142, 2)

In [692]:
item_meta = item_meta.merge(seoul_item, left_on = 'item_id', right_on = 'reference', how = 'left')

In [697]:
item_meta.loc[item_meta['index'].notnull(), 'index'] = 1

In [699]:
item_meta = item_meta.drop('reference', axis = 1).rename(columns = {'index' : 'isvalid'})

In [774]:
# 서울 호텔 개수 0~991 까지
item_meta.sort_values('isvalid').reset_index(drop = True).reset_index()['isvalid'].sum()

992.0

In [705]:
item2idx = item_meta.sort_values('isvalid').reset_index(drop = True).reset_index()[['index', 'item_id']]

In [713]:
df = log_all.loc[log_all['action_type'] == 'clickout item', ['user_id', 'reference', 'timestamp']].reset_index(drop = True).sort_values(['user_id', 'timestamp'])

In [716]:
# reference가 비어있는 row -> test로 사용할 예정
test = df.loc[df['reference'].isnull()]

In [717]:
df = df.loc[df['reference'].notnull()]

In [729]:
df.shape

(1861792, 3)

In [730]:
df = df.merge(df.groupby('user_id')['reference'].count().reset_index(), on = 'user_id', how = 'left')

In [734]:
df = df.loc[df['reference_y'] > 2][['user_id', 'reference_x']]

In [785]:
df = df.merge(item2idx, left_on = 'reference_x', right_on = 'item_id', how = 'left')

In [786]:
df

,user_id,reference_x,index,item_id
0,0001VQMGUI65,3133074,100442.0,3133074
1,0001VQMGUI65,477811,681438.0,477811
2,0001VQMGUI65,3133074,100442.0,3133074
3,0001VQMGUI65,2019467,276235.0,2019467
4,0001VQMGUI65,950829,773818.0,950829
...,...,...,...,...
1090049,ZZZSKOPW9P49,67108,66529.0,67108
1090050,ZZZSKOPW9P49,5083332,493905.0,5083332
1090051,ZZZSKOPW9P49,9094160,426647.0,9094160
1090052,ZZZSKOPW9P49,915899,391399.0,915899


In [788]:
df = df.loc[df['index'].notnull()]

In [790]:
user2idx = df['user_id'].drop_duplicates().reset_index(drop= True).reset_index()

In [791]:
final = df.merge(user2idx, on = 'user_id', how = 'left')

In [792]:
final.shape

(1089610, 5)

In [796]:
final = final[['index_y', 'index_x']]

In [797]:
final['index_x'] = final['index_x'].astype(int)

/Users/jinseok/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [800]:
final.to_csv('./test_all.txt', header = None, index = False)

In [798]:
final

,index_y,index_x
0,0,100442
1,0,681438
2,0,100442
3,0,276235
4,0,773818
...,...,...
1089605,199579,66529
1089606,199579,493905
1089607,199579,426647
1089608,199579,391399


In [799]:
final['index_x'].nunique(), final['index_y'].nunique()

(220098, 199580)